<a href="https://colab.research.google.com/github/cbunch99/Homework2/blob/main/Putting_it_together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [3]:
#import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
import pandas as pd

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/DelayedFlights.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("DelayedFlights.csv"), sep=",", header=True)
url_cities='https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/cities500.txt'
spark.sparkContext.addFile(url_cities)
df_lookup_geo = spark.read.csv(SparkFiles.get("cities500.txt"), sep="\t", header=True)

# we are going to do a lookup here as well so upload the airportCodes.csv file from you Resources directory 
df_lookup_city_name=spark.read.csv('/content/airportCodes.csv', sep=',', header=True)

In [6]:
#Create temporary views for each of our dataframes
# We are going to filter the data to US only as we create the Temp Views.

df.createOrReplaceTempView('delayed')
df_lookup_city_name.createOrReplaceTempView('lookup_city')
df_lookup_geo.createOrReplaceTempView('lookup_geo')

In [7]:
df.show()

+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|_c0|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|2008|    1|         3|        4| 2003.0|      1955| 2211.0|      2225|       

In [8]:
df_lookup_geo.show()

+---------+-------------------+-------------------+--------------------+--------+---------+-------------+------------+------------+----+-----------+-----------+-----------+-----------+----------+---------+----+--------------+-----------------+
|geonameid|               name|          asciiname|      alternatenames|latitude|longitude|feature_class|feature_code|country_code| cc2|admin1_code|admin2_code|admin3_code|admin4_code|population|elevation| dem|      timezone|modification_date|
+---------+-------------------+-------------------+--------------------+--------+---------+-------------+------------+------------+----+-----------+-----------+-----------+-----------+----------+---------+----+--------------+-----------------+
|  3038999|             Soldeu|             Soldeu|                null|42.57688|  1.66769|            P|         PPL|          AD|null|         02|       null|       null|       null|       602|     null|1832|Europe/Andorra|       2017-11-06|
|  3039154|          El 

In [9]:
df_lookup_city_name.show()

+--------------+--------------------+-----------+
|          City|             country|airportCode|
+--------------+--------------------+-----------+
|       Aalborg|             Denmark|        AAL|
|      Aalesund|              Norway|        AES|
|        Aarhus|             Denmark|        AAR|
|Abbotsford, BC|              Canada|        YXX|
|Abbotsford, BC|              Canada|        YXX|
|      Aberdeen|            Scotland|        ABZ|
|  Aberdeen, SD|                 USA|        ABR|
|       Abidjan|         Ivory Coast|        ABJ|
|   Abilene, TX|                 USA|        ABI|
|     Abu Dhabi|United Arab Emirates|        AUH|
|         Abuja|             Nigeria|        ABV|
|      Acapulco|              Mexico|        ACA|
|         Accra|               Ghana|        ACC|
|         Adana|              Turkey|        ADA|
|   Addis Ababa|            Ethiopia|        ADD|
|Adelaide, S.A.|           Australia|        ADL|
|          Aden|               Yemen|        ADE|


In [10]:
# Here is the  initial query presented to you for optimization
# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|    Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ATL|           EV|    Atlanta, GA|         33.749|       -84.38798|     31.31129|     -92.44514|   99.0| 28.81025641025641|
|   ABQ|           DL|Albuquerque, NM|       35.08449|      -106.65114|       33.749|     -84.38798|   99.0|40.401869158878505|
|   ATW|           EV|   Appleton, WI|       44.26193|       -88.41538|       33.749|     -84.38798|   99.0|30.546666666666667|
|   BWI|           WN|  Baltimore, MD|       39.29038|       -76.61219|     30.26715|     -97.74306|   95.0|12.741935483870968|
|   CAK|           EV|      Akron, OH|       41.08144|       -81.51901|       33.749|     -84.38798|   9

In [25]:
# Note the runtime
start_time = time.time()

df1 = spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.17195773124694824 seconds ---


In [26]:
df1.show()

+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|    Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ATL|           EV|    Atlanta, GA|         33.749|       -84.38798|     31.31129|     -92.44514|   99.0| 28.81025641025641|
|   ABQ|           DL|Albuquerque, NM|       35.08449|      -106.65114|       33.749|     -84.38798|   99.0|40.401869158878505|
|   ATW|           EV|   Appleton, WI|       44.26193|       -88.41538|       33.749|     -84.38798|   99.0|30.546666666666667|
|   BWI|           WN|  Baltimore, MD|       39.29038|       -76.61219|     30.26715|     -97.74306|   95.0|12.741935483870968|
|   CAK|           EV|      Akron, OH|       41.08144|       -81.51901|       33.749|     -84.38798|   9

In [27]:
#partition the largest table
df1.write.parquet('parquet_delayed',mode='overwrite')

In [30]:
p_df=spark.read.parquet('parquet_delayed')

In [31]:
p_df.createOrReplaceTempView('parquet_all')

In [32]:
start_time = time.time()
spark.sql("""
select *
from parquet_all
""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|    Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ABQ|           WN|Albuquerque, NM|       35.08449|      -106.65114|     39.29038|     -76.61219|   96.0| 9.567164179104477|
|   ABQ|           XE|Albuquerque, NM|       35.08449|      -106.65114|     30.26715|     -97.74306|   95.0| 5.610169491525424|
|   ABQ|           DL|Albuquerque, NM|       35.08449|      -106.65114|       33.749|     -84.38798|   99.0|40.401869158878505|
|   ATW|           EV|   Appleton, WI|       44.26193|       -88.41538|       33.749|     -84.38798|   99.0|30.546666666666667|
|   BWI|           WN|  Baltimore, MD|       39.29038|       -76.61219|     30.26715|     -97.74306|   9

In [ ]:
# run 2 after storing the data more appropriately and partitioning

# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()
print("--- %s seconds ---" % (time.time() - start_time))


In [34]:
df1.write.partitionBy("UniqueCarrier").mode("overwrite").parquet("delayed_partitioned")

In [35]:
p_df_p=spark.read.parquet('delayed_partitioned')

In [39]:
start_time = time.time()
spark.sql("""
select *
from p_delays_p
""").show()
print("--- %s seconds ---" % (time.time() - start_time))

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



AnalysisException: ignored

In [40]:
# Recall that the default shuffle partitions is 200.  We want to bring that down to a reasonable size for both our data and our Spark cluster
# 4 is reasonable for a free Colab 
spark.conf.set("spark.sql.shuffle.partitions", 4)

In [41]:
# Run 3 after setting the shuffle partitions to a more appropriate number
# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+-------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|  Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+-------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ABE|           OO|Allentown, PA|       40.60843|       -75.49018|     33.52066|     -86.80249|   11.0|              null|
|   ALB|           OH|   Albany, NY|       42.65258|       -73.75623|     42.35843|     -71.05977|   55.0|               0.0|
|   ALB|           WN|   Albany, NY|       42.65258|       -73.75623|     39.29038|     -76.61219|   99.0| 8.203647416413373|
|   ASE|           OO|    Aspen, CO|        39.1911|      -106.81754|       33.749|     -84.38798|   82.0|               6.0|
|   ATL|           9E|  Atlanta, GA|         33.749|       -84.38798|     33.52066|     -86.80249|   94.0|15.083333333

In [37]:
# cache your largest temporary view
# Note: when we use SparkSQL to cache a table, the table is immediately cached (no lazy evaluation), when using Pyspark it will not be cached until an action is ran.

spark.sql("cache table delayed")

DataFrame[]

In [ ]:
spark.sql("cache table lookup_city")

In [ ]:
# check that your table is cached 
spark.catalog.isCached("delayed")

In [ ]:
spark.catalog.isCached("lookup_city")

In [ ]:
# Run 4 - after caching driver table
# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df_clean = df[['Origin','CarrierDelay','UniqueCarrier','DepDelay','Dest']]

In [ ]:
usa_look_up_cities = spark.sql("""SELECT * from lookup_city where Country = 'USA' """)

In [ ]:
usa_look_up_geo = spark.sql("""SELECT name,
              latitude,
              longitude,
              admin1_code,
              country_code
              from lookup_geo where country_code = 'US' """)


In [ ]:
usa_look_up_cities.createOrReplaceTempView('lookup_city')
usa_look_up_geo.createOrReplaceTempView('lookup_geo')
df_clean.createOrReplaceTempView('delayed')
spark.sql("cache table delayed")
spark.sql("cache table lookup_geo")

In [ ]:
spark.sql(""" SELECT  * from delayed a
              inner join lookup_city b
               on a.Origin=b.airportCode
               """).show()

In [ ]:
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.UniqueCarrier,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
b.City as Dest_City,
geo.latitude as Dest_latitude,
geo.longitude as Dest_longitude,
a.CarrierDelay
     from  delayed a 
     inner join lookup_city b
    on a.Origin=b.airportCode
    inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code))
    select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
  """).show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Run 5 - caching one of the lookups
#Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
spark.sql("uncache table delayed")
spark.sql("uncache table lookup_geo")

In [ ]:

#Verify that the table is no longer cached
if spark.catalog.isCached("delayed") or spark.catalog.isCached("lookup_geo"):
  print("a table is till cached")
else:
  print("all clear")